<h1>Report on boosting<span class="tocSkip"></span></h1>

**High-dimensional statistics** lecture by [Anne Gégout-Petit](http://www.iecl.univ-lorraine.fr/~Anne.Gegout/).

Authors: [Sylvain Combettes](https://sylvaincom.github.io/) and [Lucas Lherbier](https://www.linkedin.com/in/lucas-lherbier/).

Last update: Nov 24th, 2019.

---
This notebook completes our report on boosting. It contains a few implementations of the methods that were studied. In particular, we compare several classifiers on several datasets (for classication). All functions (except XGBoost) and datasets come from scikit-learn.

<div class="alert alert-info"><h4>README<span class="tocSkip"></span></h4><p>
The best way to open this Jupyter Notebook is to use the table of contents from the extensions called <code>nbextensions</code>. See <a href="https://towardsdatascience.com/4-awesome-tips-for-enhancing-jupyter-notebooks-4d8905f926c5">4 Awesome Tips for Enhancing Jupyter Notebooks</a> by George Seif.
    
The Python version is 3.7.3.
</p></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Defining-a-benchmarking-function-of-scores-and-processing-time-on-several-classifiers" data-toc-modified-id="Defining-a-benchmarking-function-of-scores-and-processing-time-on-several-classifiers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Defining a benchmarking function of scores and processing time on several classifiers</a></span></li><li><span><a href="#Launching-our-benchmark-on-several-simple-datasets-(for-classification)" data-toc-modified-id="Launching-our-benchmark-on-several-simple-datasets-(for-classification)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Launching our benchmark on several simple datasets (for classification)</a></span><ul class="toc-item"><li><span><a href="#Iris-plants-dataset" data-toc-modified-id="Iris-plants-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Iris plants dataset</a></span></li><li><span><a href="#Optical-recognition-of-handwritten-digits-dataset" data-toc-modified-id="Optical-recognition-of-handwritten-digits-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Optical recognition of handwritten digits dataset</a></span></li><li><span><a href="#Wine-recognition-dataset" data-toc-modified-id="Wine-recognition-dataset-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Wine recognition dataset</a></span></li><li><span><a href="#Breast-cancer-wisconsin-(diagnostic)-dataset" data-toc-modified-id="Breast-cancer-wisconsin-(diagnostic)-dataset-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Breast cancer wisconsin (diagnostic) dataset</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#Launching-our-benchmark-on-a-large-dataset-(for-classification)" data-toc-modified-id="Launching-our-benchmark-on-a-large-dataset-(for-classification)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Launching our benchmark on a large dataset (for classification)</a></span></li><li><span><a href="#XGBoost-on-the-iris-dataset-(without-the-scikit-learn-API)" data-toc-modified-id="XGBoost-on-the-iris-dataset-(without-the-scikit-learn-API)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>XGBoost on the iris dataset (without the scikit-learn API)</a></span></li><li><span><a href="#Further-work" data-toc-modified-id="Further-work-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Further work</a></span></li></ul></div>

<h3>Imports<span class="tocSkip"></span></h3>

In [1]:
import pandas as pd
import numpy as np

from time import process_time
import datetime

from sklearn import datasets, model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb
from xgboost import XGBClassifier

# Defining a benchmarking function of scores and processing time on several classifiers

We are going to compare 4 classifiers from scikit-learn:
- [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
- [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

and [XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn) (a Python library on itself). However, XGBoost has a Python API for scikit-learn. For example, we can use scikit-learn's cross-validation on XGBoost.

For each of the previous classifiers, we choose `n_estimators=50` and `max_depth=2` (when it applies). Thus, we do not tune the hyper-parameters.

We are going to define a Python function that will return the benchmark of scores and processing time for each method. This function will be based on scikit-learn:
- [Model selection and evaluation](https://scikit-learn.org/stable/model_selection.html)
- [sklearn.model_selection.cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

We choose 10-fold cross validation.

First, we define an intermediate function:

In [2]:
def score_and_time(model, X_dataset, y_dataset, cv):
    """
    This function returns a list with the scores and processing times of model.
    The scores are calculated with cross_val_score (with K-Fold equal to cv).
    There are no hyper-parameters.
    """
    
    t_start = process_time()
    scores = model_selection.cross_val_score(model, X_dataset, y_dataset, cv=cv)
    t_stop = process_time()
    part_l = [round(scores.mean(), 3), round(scores.std()*2, 3), datetime.timedelta(seconds=t_stop-t_start)]
    
    return part_l

Second, we define the benchmarking function itself:

In [3]:
def ml_benchmark(X_dataset, y_dataset, cv):
    """
    This function returns a pandas dataframe with the scores and processing times of several classifiers
        applied to X_dataset and y_dataset.
    The scores are calculated with cross_val_score (with K-Fold equal to cv).
    """
    
    print('The shape of X_dataset is:', X_dataset.shape)
    print('The shape of y_dataset is:', y_dataset.shape)
    print('The set of values of y_dataset is:', set(y_dataset))
     
    rows_name = ["DecisionTreeClassifier", "RandomForestClassifier", "AdaBoostClassifier",
                 "GradientBoostingClassifier", "XGBoost"]
    
    columns_name = ['Approx. mean of scores', 'Approx. variance of scores', 'Processing time']
    
    l = []
    
    model = DecisionTreeClassifier(random_state=0)
    l.append(score_and_time(model, X_dataset, y_dataset, cv))
        
    model = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=0)
    l.append(score_and_time(model, X_dataset, y_dataset, cv))
    
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=50, algorithm='SAMME',
                               random_state=0)
    l.append(score_and_time(model, X_dataset, y_dataset, cv))
        
    model= GradientBoostingClassifier(n_estimators=50, max_depth=2)
    l.append(score_and_time(model, X_dataset, y_dataset, cv))
    
    model = xgb.XGBClassifier(n_estimators=50, max_depth=2, random_state=0)
    l.append(score_and_time(model, X_dataset, y_dataset, cv))
    
    out = pd.DataFrame(l, index = rows_name, columns = columns_name)
    
    return out

# Launching our benchmark on several simple datasets (for classification)

We are going to launch our benchmark on several simple [datasets](https://scikit-learn.org/stable/datasets/index.html#toy-datasets) from scikit-learn. These datasets are all intented for classification (and not regression):
- [Iris plants dataset](https://scikit-learn.org/stable/datasets/index.html#iris-plants-dataset)
- [Optical recognition of handwritten digits dataset](https://scikit-learn.org/stable/datasets/index.html#optical-recognition-of-handwritten-digits-dataset)
- [Wine recognition dataset](https://scikit-learn.org/stable/datasets/index.html#wine-recognition-dataset)
- [Breast cancer wisconsin (diagnostic) dataset](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-wisconsin-diagnostic-dataset)

A description of these simple datasets is provided in the hyperlinks above.

Indeed, scikit-learn comes with a few small standard datasets that do not require to download any file from some external website. These datasets are useful to quickly illustrate the behavior of the various algorithms implemented in scikit-learn. They are however often too small to be representative of real world machine learning tasks.

## Iris plants dataset

In [4]:
dataset = datasets.load_iris()
X_dataset = dataset.data
y_dataset = dataset.target

df_benchmark_1 = ml_benchmark(X_dataset, y_dataset, 10)
df_benchmark_1_sorted = df_benchmark_1.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_1_sorted

The shape of X_dataset is: (150, 4)
The shape of y_dataset is: (150,)
The set of values of y_dataset is: {0, 1, 2}


,Approx. mean of scores,Approx. variance of scores,Processing time
DecisionTreeClassifier,0.960,0.088,00:00:00.008216
XGBoost,0.960,0.088,00:00:00.108430
RandomForestClassifier,0.953,0.120,00:00:00.263899
AdaBoostClassifier,0.953,0.085,00:00:00.329929
GradientBoostingClassifier,0.953,0.085,00:00:00.474944


## Optical recognition of handwritten digits dataset

In [5]:
dataset = datasets.load_digits()
X_dataset = dataset.data
y_dataset = dataset.target

df_benchmark_2 = ml_benchmark(X_dataset, y_dataset, 10)
df_benchmark_2_sorted = df_benchmark_2.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_2_sorted

The shape of X_dataset is: (1797, 64)
The shape of y_dataset is: (1797,)
The set of values of y_dataset is: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


,Approx. mean of scores,Approx. variance of scores,Processing time
GradientBoostingClassifier,0.913,0.072,00:00:12.257602
XGBoost,0.910,0.061,00:00:07.209754
AdaBoostClassifier,0.855,0.059,00:00:02.023138
DecisionTreeClassifier,0.830,0.077,00:00:00.161670
RandomForestClassifier,0.791,0.103,00:00:00.485181


## Wine recognition dataset

In [6]:
dataset = datasets.load_wine()
X_dataset = dataset.data
y_dataset = dataset.target

df_benchmark_3 = ml_benchmark(X_dataset, y_dataset, 10)
df_benchmark_3_sorted = df_benchmark_3.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_3_sorted

The shape of X_dataset is: (178, 13)
The shape of y_dataset is: (178,)
The set of values of y_dataset is: {0, 1, 2}


,Approx. mean of scores,Approx. variance of scores,Processing time
RandomForestClassifier,0.973,0.072,00:00:00.282316
AdaBoostClassifier,0.972,0.075,00:00:00.442636
GradientBoostingClassifier,0.963,0.157,00:00:00.608170
XGBoost,0.961,0.100,00:00:00.194422
DecisionTreeClassifier,0.895,0.144,00:00:00.011798


## Breast cancer wisconsin (diagnostic) dataset

In [7]:
dataset = datasets.load_breast_cancer()
X_dataset = dataset.data
y_dataset = dataset.target

df_benchmark_4 = ml_benchmark(X_dataset, y_dataset, 10)
df_benchmark_4_sorted = df_benchmark_4.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_4_sorted

The shape of X_dataset is: (569, 30)
The shape of y_dataset is: (569,)
The set of values of y_dataset is: {0, 1}


,Approx. mean of scores,Approx. variance of scores,Processing time
AdaBoostClassifier,0.970,0.035,00:00:01.516546
XGBoost,0.963,0.057,00:00:00.356844
GradientBoostingClassifier,0.960,0.056,00:00:00.451752
RandomForestClassifier,0.946,0.070,00:00:00.383812
DecisionTreeClassifier,0.918,0.065,00:00:00.062199


## Conclusion

Based on the previous results, we can write the following ranking based on the mean of the scores on the four datasets:

In [8]:
df_benchmark_total = (df_benchmark_1+df_benchmark_2+df_benchmark_3+df_benchmark_4) # we sum the four benchmarks
df_benchmark_total = df_benchmark_total.divide(4) # we divide by the number of datasets to get the mean
df_benchmark_total_sorted = df_benchmark_total.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_total_sorted

,Approx. mean of scores,Approx. variance of scores,Processing time
XGBoost,0.94850,0.07650,00:00:01.967362
GradientBoostingClassifier,0.94725,0.09250,00:00:03.448117
AdaBoostClassifier,0.93750,0.06350,00:00:01.078062
RandomForestClassifier,0.91575,0.09125,00:00:00.353802
DecisionTreeClassifier,0.90075,0.09350,00:00:00.060970


Thus, our ranking corresponds to what is commonly stated:
1. XGBoost
2. GradientBoostingClassifier
3. AdaBoostClassifier
4. RandomForestClassifier
5. DecisionTreeClassifier

In particular, `XGBoost` is much faster than `GradientBoostingClassifier`.

Let us note that the datasets we used are too small to be representative of real world machine learning tasks: we can expect that the range of scores and processing time will increase. For example, with a larger dataset, we can expect `DecisionTreeClassifier` to perform much worse while `XGBoost` would perform not so badly: that it the case for the `digits` dataset that has more samples than the others. In the next section, we are going to address a larger dataset.

# Launching our benchmark on a large dataset (for classification)

We generate our classification dataset with scikit-learn's `make_classification` function.

We try to be in the context of the _Statistics for high-dimensional data_ lecture by choosing `n_samples=500` and `n_features=1000` so that $n \ll p$.

More information can be found on scikit-learn's [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html).

In [9]:
dataset = make_classification(n_samples=500, n_features=1000, n_informative=20, n_classes=5,
                              flip_y=0.001, class_sep=2, n_clusters_per_class=5, random_state=0)
X_dataset = dataset[0]
y_dataset = dataset[1]

In [10]:
df_benchmark_large = ml_benchmark(X_dataset, y_dataset, 10)
df_benchmark_large_sorted = df_benchmark_large.sort_values(by=['Approx. mean of scores'], ascending=False)
df_benchmark_large_sorted

The shape of X_dataset is: (500, 1000)
The shape of y_dataset is: (500,)
The set of values of y_dataset is: {0, 1, 2, 3, 4}


,Approx. mean of scores,Approx. variance of scores,Processing time
XGBoost,0.450,0.128,00:00:36.693923
GradientBoostingClassifier,0.417,0.112,00:00:44.510023
AdaBoostClassifier,0.416,0.086,00:00:35.098607
RandomForestClassifier,0.352,0.151,00:00:00.943000
DecisionTreeClassifier,0.317,0.151,00:00:02.128367


Note: the scores are bad but here we focus on comparing the models between them

According to the previous results, we have the same ranking as before but the range in scores and processing time is higher on large datasets than smaller ones:

In [11]:
scores = df_benchmark_total_sorted['Approx. mean of scores']
perc_inc = round((scores[0]-scores[4])/(scores[4])*100, 2) # percentage increase of scores
print('On the small datasets, the score increased from DecisionTree to XGBoost by {}%.'.format(perc_inc))

On the small datasets, the score increased from DecisionTree to XGBoost by 5.3%.


In [12]:
scores = df_benchmark_large_sorted['Approx. mean of scores']
perc_inc = round((scores[0]-scores[4])/(scores[4])*100, 2) # percentage increase of scores
print('On the large dataset, the score increased from DecisionTree to XGBoost by {}%.'.format(perc_inc))

On the large dataset, the score increased from DecisionTree to XGBoost by 41.96%.


On large datasets (that are harder to predict), it is more obvious that `XGBoost` performs much better than `DecisionTreeClassifier`.

# XGBoost on the iris dataset (without the scikit-learn API)

Unfortunately, XGBoost is not on scikit-learn, thus we apply it in a different section.
The goal of this section is run XGBoost without using the scikit-learn API in order to understand the `DMatrix` format.
We used the following tutorial: [A Beginner’s guide to XGBoost](https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7).

First of all, we load and prepare the data:

In [13]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In order for XGBoost to be able to use our data, we’ll need to transform it into a specific format called `DMatrix` that XGBoost can handle:

In [14]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

We define our XGBoost model:

In [15]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3}

steps = 20  # The number of training iterations

We train and we test our model:

In [16]:
model = xgb.train(param, D_train, steps)

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9761904761904763
Recall = 0.9444444444444445
Accuracy = 0.9666666666666667


# Further work

The following scikit-learn tutorials are interesting to read:
- [Classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py)
- [Discrete versus Real AdaBoost](https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py)
- [Two-class AdaBoost](https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_twoclass.html)
- [Multi-class AdaBoosted Decision Trees](https://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_multiclass.html)
- [Plot the decision surfaces of ensembles of trees on the iris dataset](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_iris.html)

---
Back to [top](#top).